In [3]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# key joints for a sit to stand
# Hip joints: 23,24 (left, right hips)
# Knee joints: 25,26 (left, right knees)
# Ankle joints: 27,28 (left, right ankles)


In [4]:
for landmark in mp_pose.PoseLandmark:
    if (landmark.name == "LEFT_HIP" or
        landmark.name == "RIGHT_HIP" or
        landmark.name == "LEFT_KNEE" or
        landmark.name == "RIGHT_KNEE" or
        landmark.name == "LEFT_ANKLE" or
        landmark.name == "RIGHT_ANKLE"):
        print(landmark.name, landmark.value)

LEFT_HIP 23
RIGHT_HIP 24
LEFT_KNEE 25
RIGHT_KNEE 26
LEFT_ANKLE 27
RIGHT_ANKLE 28


In [5]:
# b is the midpoint of a and c (e.g. left hip, left elbow and left shoulder)
# our case will be left-hip, left-knee and left-ankle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

        
    return angle 

In [12]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Curl counter variables
counter = 0
stage = None
frame_count = 0
confirm_frames = 5
stage_counter = 0

# for storing the max angle achieved in a rep
max_angle_per_rep = 0
last_angle = 0

# for optical flow postprocessing
hip_history = []
knee_history = []
ankle_history = []

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture("../test/CST_self2.mp4")
    while cap.isOpened():
        # how to read from video

        ret, frame = cap.read()
        
        if not ret:
            logging.warning("No frame captured from the video source.")
            break
        logging.info("Frame captured and processing started.")


        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            # Check if landmarks are detected
            if not results.pose_landmarks:
                logging.warning("No pose landmarks detected.")
                continue
            
            landmarks = results.pose_landmarks.landmark


            # Get coordinates
            hip = [
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
            ]
            ankle = [
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
            ]
            knee = [
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
            ]
            hip_history.append(hip)
            knee_history.append(knee)
            ankle_history.append(ankle)

            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            logging.info(f"Calculated knee angle: {angle}")

            # Update max angle for the current rep
            if angle > max_angle_per_rep:
                max_angle_per_rep = angle

            # Check if angle starts to decrease
            if last_angle > angle:
                max_angle_achieved = max_angle_per_rep
            last_angle = angle
            
            
            # Visualize angle
            cv2.putText(
                image,
                str(f"knee angle: {angle}"),
                tuple(np.multiply(knee, [640, 480]).astype(int)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            # CST counter logic
            if stage is None:
                # Determine the initial stage based on the first frame's angle
                if angle > 135:
                    stage = "up"
                    logging.info("Initial stage set to up.")

                else:
                    stage = "down"
                    logging.info("Initial stage set to down.")


            if stage == "down" and angle > 135:
                stage_counter += 1
                if stage_counter >= confirm_frames:
                    stage = "up" 
                    stage_counter = 0
                    counter += 1  # Increment counter on transitioning to "up"
                    logging.info(f"Transitioned to up. Total reps: {counter}")
                    max_angle_per_rep = 0  # Reset max angle for the new repetition

            elif stage == "up" and angle < 90:
                stage_counter += 1
                if stage_counter >= confirm_frames:
                    stage = "down"
                    stage_counter = 0  # Reset the stage counter after confirming the stage
                    logging.info("Transitioned to down.")
        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Rep data
        cv2.putText(
            image,
            "REPS",
            (15, 12),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
            cv2.LINE_AA,
        )
        cv2.putText(
            image,
            str(counter),
            (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            2,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        # Stage data
        cv2.putText(
            image,
            "STAGE",
            (65, 12),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 0),
            1,
            cv2.LINE_AA,
        )
        cv2.putText(
            image,
            stage,
            (60, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            2,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        cv2.imshow("Mediapipe Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    print("video ended")
    cv2.destroyAllWindows()
    cap.release()

I0000 00:00:1716129011.137731 13371713 gl_context.cc:357] GL version: 2.1 (2.1 ATI-5.2.4), renderer: AMD Radeon Pro 5500M OpenGL Engine
2024-05-19 22:30:11,278 - INFO - Frame captured and processing started.
/Users/brennanlee/Desktop/MediaPipePoseEstimation/.venv/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
2024-05-19 22:30:11,500 - INFO - Calculated knee angle: 83.88986245454805
2024-05-19 22:30:11,502 - INFO - Initial stage set to down.
2024-05-19 22:30:11,636 - INFO - Frame captured and processing started.
2024-05-19 22:30:11,676 - INFO - Calculated knee angle: 83.9222534331913
2024-05-19 22:30:11,734 - INFO - Frame captured and processing started.
2024-05-19 22:30:11,767 - INFO - Calculated knee angle: 83.42672399432219
2024-05-

video ended
